# ELS, Internalizing problems and cardio-metabolic health in childhood


Hello there, here is the script for creating the figures for the paper on the relationship between prenatal and postnatal stress and the multimorbidity between internalizing symptoms and cardio-metabolic health measured in 9-10 year-old children from the Generation R study (https://generationr.nl/). 

##### First, let's get the dependencies we need

In [ ]:
# getting to the data
import os.path
# data storage and handling
import pyreadr
import pandas as pd
import numpy as np
# plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatch
import seaborn as sns
import ptitprince as pt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
# stats
import scipy
# ignore warnings 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Point to the raw data
pathtodata = input("Enter the path to data: ") # mind the slashes e.g. /Users/Serena/Desktop/Data/

if os.path.exists(pathtodata) == False:
    pathtodata = input("Not quite, try again: ")

In [ ]:
# Here we define some key functions that are going to make extracting datasets and plotting them easier

def read_and_clean(file, fileformat = "rsd", path = pathtodata):
    '''This function reads rsd / SPSS format data files into pandas dataframes and sets the IDC column as index.
       Once I am at it, I get rid of empty rows as well'''
    
    if fileformat != "rsd":
        # read in the .sav file
        df = pd.read_spss(pathtodata + file)
    else: 
        # read in the .rsd file
        ds = pyreadr.read_r(pathtodata + file)
        # result is a dictionary where keys are the name of objects and the values are python
        # objects. In the case of Rds there is only one object with None as key.
        df = ds[None] # extract the pandas data frame 
        
    # set IDC to be the index rather than a column
    df.IDC = pd.to_numeric(df.IDC, downcast = 'integer')
    df.index = df.IDC
    df = df.drop(['IDC'], axis = 1)
    
    # drop children with no available data 
    df = df.dropna(how = 'all')
    
    return df

# ------------------------------------------------------------------------------------------------------- #

def stress2D(ax, outcome, stressor):
    '''This function creates classic (2D) regression graphs: i.e. a scatterplot of the exposure vs outcome 
       + regression line illustrating the main effect of exposure on outcome '''
    
    # outcome variable names and betas dataframe
    out = 'intern_score_z' if outcome == 'Internalizing' else 'fat_mass_z'
    bs = bs_int if outcome == 'Internalizing' else bs_fat
    # exposure variable names and betas dataframe
    exp = 'prenatal_stress_z' if stressor == 'Prenatal stress'else 'postnatal_stress_z'
    beta_pos = 1 if stressor == 'Prenatal stress' else 2
    
    # Define range of values for axis limits 
    out_mn = np.min(np.min(d[['intern_score_z', 'fat_mass_z']], axis = 0))
    out_mx = np.max(np.max(d[['intern_score_z', 'fat_mass_z']], axis = 0))
    # Set axis limits 
    ax.set_xlim([mn[1] - 0.1, mx[1] + 0.1])  # stress boundaries
    ax.set_ylim([out_mn - 0.1, out_mx + 0.1]) # z score boundaries
    # Scatterplot
    ax.plot(d[exp], d[out], 'o', color = 'k', alpha = 0.1)
    # Regression line
    ax.plot(d[exp], bs[beta_pos]*d[exp] + bs[0], color = cols[0] if exp == 'prenatal_stress_z' else cols[1], lw = 2.5)
    # Add a grid 
    ax.grid()
    # Set labels for the axes
    ax.set_xlabel(stressor, fontsize = 17, fontweight = 'bold')
    ax.set_ylabel(outcome, fontsize = 17, fontweight = 'bold')
    # ax.set_title(stressor, fontsize = 20, fontweight = 'bold')
# ------------------------------------------------------------------------------------------------------- #   

def stress3D(ax, outcome):
    ''' This function creates a 3D model rapresentation: i.e. a scatterplot of the exposure vs outcome 
       + fitted regression surface illustrating the interaction between two predictors as well as the 
       respective main effects (colored lines)'''
    
    # outcome variable names, betas dataframe and fitted surface
    var = 'intern_score_z' if outcome == 'Internalizing' else 'fat_mass_z'
    bs = bs_int if outcome == 'Internalizing' else bs_fat
    Z = Zi if outcome == 'Internalizing' else Zf
    
    # Set axis limits for the outcome (z axis)
    ax.set_zlim([-1, 6])
    # 3D Scatterplot
    ax.scatter(d.prenatal_stress_z, d.postnatal_stress_z, d[var], s = 20, alpha = 0.1, facecolors='none', edgecolors='k')
    # Regression surface
    ax.plot_surface(X, Y, Z, rstride = 2, cstride = 2, cmap='magma', alpha = 0.4) # edgecolor = 'yellow', color = 'yellow', alpha = 0.3)
    # Main effect lines 
    ax.plot(d.prenatal_stress_z, np.zeros(d.shape[0]), bs[1]*d.prenatal_stress_z + bs[0], c = cols[0], lw = 2.5) 
    ax.plot(np.zeros(d.shape[0]), d.postnatal_stress_z, bs[2]*d.postnatal_stress_z + bs[0], c = cols[1], lw = 2.5) 
    # Set labels for the axes
    ax.set_xlabel('Prenatal ELS', fontsize = 17, fontweight = 'bold')
    ax.set_ylabel('Postnatal ELS', fontsize = 17, fontweight = 'bold')
    ax.set_zlabel(outcome, fontsize = 17, fontweight = 'bold')
    # Invert x axis to have low values of both variables in the front corner
    ax.invert_xaxis()
    # Adjust the plot size to fit the 2D plots 
    ax.dist = 8 # higher numbers, smaller plots
    

#### THEN, GET ALL THE DATA WE NEED IN MEMORY

In [ ]:
# GET THE DATASET (i.e., 30th imputation)
data = read_and_clean('ELSPCM_imputed.rds')
# print(data.columns.tolist())

# Select only some useful columns
d = data[['pre_life_events', 'pre_contextual_risk', 'pre_personal_stress', 'pre_interpersonal_stress', 
          'post_life_events', 'post_contextual_risk', 'post_parental_risk', 'post_interpersonal_risk', 'post_direct_victimization',
          'prenatal_stress', 'postnatal_stress', 'prenatal_stress_z', 'postnatal_stress_z', 
          'intern_score_z', 'fat_mass_z', 'risk_groups', 'sex', 'age_child']]
# d.head(3)

# GET THE POOLED ESTIMATES 
# extract the summary results created in script 4-Regressions.R
int_min = pd.read_excel(pathtodata+'Results.xlsx', sheet_name="1.intern_min")
fat_min = pd.read_excel(pathtodata+'Results.xlsx', sheet_name="3.fatmas_min")
grp_min = pd.read_excel(pathtodata+'Results.xlsx', sheet_name="5.riskgrp_min")

<h2><center>Ok, let's plot!</center></h2>

### MODEL (1) AND (2) : Internalizing & Android fat mass linear associations

In [ ]:
# To prepare for visualization, we constract a regular grid covering the domain of the data

# Create data array with X = all predictors
# Note: .values converts pandas series object into array so we can use linalg on it later
pred = d[['prenatal_stress_z', 'postnatal_stress_z', 'sex', 'age_child']].values 

# Obtain ranges of prenatal and postnatal stress
mn = np.min(pred, axis=0); mx = np.max(pred, axis=0)
# Create a 20x20 grid within stress ranges 
X,Y = np.meshgrid(np.linspace(mn[0], mx[0], 20), np.linspace(mn[1], mx[1], 20))
XX = X.flatten()
YY = Y.flatten()
    
# Define the prediction matrix (i.e., 1st model adjusted for age and sex)
A = np.c_[np.ones(pred.shape[0]), pred, np.prod(pred[:,:2], axis=1)]

# Get the betas from the pooled analysis in script 4-Regressions.R
bs_int = np.array(int_min['estimate'])
bs_fat = np.array(fat_min['estimate'])

# Evaluate it on a grid
Zi = np.dot(np.c_[np.ones(XX.shape), XX, YY, XX*YY], bs_int[[0,1,2,5]]).reshape(X.shape)
Zf = np.dot(np.c_[np.ones(XX.shape), XX, YY, XX*YY], bs_fat[[0,1,2,5]]).reshape(X.shape)

# ~~~~~~ OPTIONAL ~~~~~~~~
# Check: run the regression again (best-fit linear plane, 1st order) on the imputed dataset we plot
# Ci,_,_,_ = scipy.linalg.lstsq(A, d['intern_score_z']) # Internalizing
# Cf,_,_,_ = scipy.linalg.lstsq(A, d['fat_mass_z']) # Fat mass

# Compute R2 (1 - SSres / SStot)
# r2_int = 1 - (((d['intern_score_z'] - np.dot(A,bs_int))**2).sum()) / (((d['intern_score_z'] - d['intern_score_z'].mean())**2).sum()) 
# r2_fat = 1 - (((d['fat_mass_z'] - np.dot(A,bs_fat))**2).sum()) / (((d['fat_mass_z'] - d['fat_mass_z'].mean())**2).sum()) 


In [ ]:
# General settings
plt.style.use('default')

# Set up a figure for the plots
f = plt.figure(figsize = (20, 12))
# Define subplots 
ax1 = f.add_subplot(231); ax2 = f.add_subplot(232); ax3 = f.add_subplot(233, projection='3d')
ax4 = f.add_subplot(234); ax5 = f.add_subplot(235); ax6 = f.add_subplot(236, projection='3d')

# Set figure title
f.suptitle('Generation R', fontsize = 35, fontweight = 'bold')

# Define colors for the main effect lines 
cols = ["crimson", "blue"]

stress2D(ax1, 'Internalizing', 'Prenatal stress')
stress2D(ax2, 'Internalizing', 'Postnatal stress')
stress3D(ax3, 'Internalizing')
stress2D(ax4, 'Fat mass', 'Prenatal stress')
stress2D(ax5, 'Fat mass', 'Postnatal stress')
stress3D(ax6, 'Fat mass')

plt.subplots_adjust(hspace=0.4, wspace=0.2)

# plt.savefig("mod1-2",dpi = 400, edgecolorcolor = "white", facecolor = "white", pad_inches=0.1, frameon=True)

### GROUP DESCRIPTIVES: Internalizing vs. Android fat mass & pie chart

In [ ]:
# Create a variable for risk group with strings instead of dummies
d["Risk"] = ["Healthy" if  x == 0 
             else "High Internalizing" if x == 1 
             else "High fat mass" if x == 2 
             else "Multimorbid" 
                   for x in pd.to_numeric(d['risk_groups'])]

d['Risk'].value_counts()

In [ ]:
# Set up a high definition figure for the plots 
fig = plt.figure(figsize=(20,20))

# Add a gridspec with 3 rows and 3 columns and a ratio of 1 to 8 to 4 between the size of the marginal and main axes.
# Also adjust the subplot parameters for a square plot.
gs = fig.add_gridspec(3, 3,  width_ratios=(8, 1, 4), height_ratios=(1, 8, 5),
                      left = 0.1, right = 0.9, bottom = 0.1, top = 0.9, wspace = 0.05, hspace = 0.05)

# Main scatterplot ax
ax = fig.add_subplot(gs[1, 0])
# Histograms axes
ax_histx = fig.add_subplot(gs[0, 0], sharex = ax); ax_histy = fig.add_subplot(gs[1, 1], sharey = ax)
# Pie axes
ax_pie =  fig.add_subplot(gs[1, 2])

# General settings
plt.style.use('bmh')
lims = [-1.2, 7.5] # for the axes
colors = { "Healthy":'seagreen', "High Internalizing" :'royalblue', "High fat mass": 'orange', "Multimorbid":'indianred'}

ax_histx.set_title("Risk groups - Generation R\n", fontsize = 30, fontweight = 'bold')

# -------------------------------------- SCATTER PLOT ---------------------------------------- #
grouped = d.groupby('Risk')
for key, group in grouped:
    group.plot(ax = ax, kind = 'scatter', x = 'intern_score_z', y = 'fat_mass_z', s = 200, marker = 'o',
               label = key, color = colors[key], alpha = 0.6)
# Axes labels 
ax.set_ylabel("Android Fat Mass (z-score)", fontsize = 26, fontweight = 'bold'); ax.set_ylim(lims)
ax.set_xlabel("\nInternalizing Symptoms (z-score)", fontsize = 26, fontweight = 'bold'); ax.set_xlim(lims)

# adjust tick size
[tk.label.set_fontsize(20) for tk in ax.yaxis.get_major_ticks()]
[tk.label.set_fontsize(20) for tk in ax.xaxis.get_major_ticks()]

# adjust legend order
handles, labels = ax.get_legend_handles_labels()
order = [2, 0, 3, 1]
ax.legend([handles[idx] for idx in order],[labels[idx] for idx in order], 
           loc = 'upper right', fontsize = 26, ncol = 2)

# -------------------------------------- HISTOGRAMS  -------------------------------------- #
h1 = sns.distplot(d['intern_score_z'], hist = True, kde = True, ax = ax_histx,
                 kde_kws = {'shade': True, 'linewidth': 3}, color = 'darkgrey')
h2 = sns.distplot(d['fat_mass_z'], hist = True, kde = True, ax = ax_histy, vertical = True,
                 kde_kws = {'shade': True, 'linewidth': 3}, color = 'darkgrey')
# remove all ticks and labels
h1.set(yticklabels=[]); h1.set(ylabel=None)
h2.set(xticklabels=[]); h2.set(xlabel=None)


# -------------------------------------- PIE CHART  -------------------------------------- #
plt.rcParams['font.size'] = 22

labels_pie = ['Healthy', 'High Fat mass', 'High Internalizing', 'Multimorbid']
colors_pie = ['seagreen', 'orange', 'royalblue', 'indianred']
sizes = [(x / d.shape[0] * 100) for x in d['Risk'].value_counts()] # calculate percent of total sample

ax_pie.pie(sizes, colors = colors_pie, autopct='%1.1f%%', pctdistance = 0.75, counterclock = False,
                         shadow = True, startangle = 90)

ax_pie.legend(labels_pie, loc = "lower right", fontsize = 22)

# Equal aspect ratio ensures that pie is drawn as a circle
ax_pie.axis('equal')

# plt.savefig("scatter-group",dpi = 400, edgecolorcolor = "white", facecolor = "white", pad_inches=0.1, frameon=True)

In [ ]:
# To build the cloudrain plot we need to stacck pre and postnatal stress in long format 
stack = pd.concat([d['prenatal_stress_z'], d['postnatal_stress_z']], axis=0).to_frame()
stack.columns = ['stress'] # rename column
stack['period'] = ['prenatal']*d.shape[0] + ['postnatal']*d.shape[0] # add column to specify period 
stack['groups'] = pd.concat([d["Risk"], d["Risk"]], axis=0) # add column with group codes

# General settings
plt.style.use('default')
pal = ['purple','teal']

# construct a figure 
f, ax = plt.subplots(figsize=(15, 15))

# Plot those clounds 
pt.RainCloud(x = stack["groups"], y = stack["stress"], hue = stack["period"], data = stack, palette = pal, 
             bw = .12, width_viol = .7, ax = ax, orient = 'h', alpha = .5, dodge = True, pointplot = True, 
             move = .2, order = ["Healthy", "High fat mass", "High Internalizing", "Multimorbid"])

# add a legend
ax.legend(handles = [mpatch.Patch(color=pal[0], alpha = .9, label = 'Prenatal ELS'),
                     mpatch.Patch(color=pal[1], alpha = .9, label = 'Postnatal ELS')], fontsize = 17)
# adjust size tick labels 
ax.set_yticklabels(['Healthy', 'High \nFat mass', 'High \nInternalizing', 'Multimorbid'], fontsize = 14, fontweight = 'bold')
ax.set_xticklabels(list(range(-2, 7)), fontsize = 14)
# adjust axis labels
ax.set_ylabel("Risk groups", fontsize = 17); 
ax.set_xlabel("Cumulative stress (z-score)", fontsize = 17, fontweight = 'bold'); 

ax.set_title('Generation R', fontsize = 35, fontweight = 'bold')

# plt.savefig("groupsrain",dpi = 400, edgecolorcolor = "white", facecolor = "white", pad_inches=0.1, frameon=True)

### MODEL (3): GROUP LOGISTIC REGRESSION

In [ ]:
# Conditional kernel density estimate

# Set up a figure for the plots
f, (ax1, ax2) = plt.subplots(1, 2, figsize = (22, 8))

# General settings
f.suptitle('Generation R\n', fontsize = 35, fontweight = 'bold')

def cond_density(exp, ax, labelx, labely = 'Density\n', leg = False):
    # Plot the distribution of risk groups, conditional on stress expsosure
    with sns.axes_style('whitegrid'):
        sns.kdeplot(data = d, x = exp, hue = "Risk", ax = ax, hue_order = reversed(colors.keys()), alpha = 0.9,
                    multiple = "fill", bw_method = 0.7, clip = (-1, 7), palette = colors, legend = leg)
        ax.set_ylabel(labely, fontsize = 24, fontweight = 'bold')
        ax.set_xlabel(labelx, fontsize = 24, fontweight = 'bold')
        ax.set_yticklabels([0., 0.2, 0.4, 0.6, 0.8, 1.], fontsize = 20)
        ax.set_xticklabels(list(range(-1, 8)), fontsize = 20)
        ax.grid(True, color = 'k', alpha = 0.8)
        if leg:
            ax.legend(labels = colors.keys(), fontsize = 24, bbox_to_anchor=(1.7, 1), edgecolor="none")
            ax.yaxis.label.set_visible(False)

cond_density("prenatal_stress_z", ax1, '\nPrenatal stress (z-score)')
cond_density("postnatal_stress_z", ax2, '\nPostnatal stress (z-score)', leg = True)

# plt.savefig("groupdensity",dpi = 400, edgecolorcolor = "white", facecolor = "white", pad_inches=0.1, frameon=True)

In [ ]:
# Drop the column with categorical data we created earlier
dmlr = d.drop('Risk', axis = 1)

# create three datasets including only reference and comparison group 
reg1 = dmlr.loc[dmlr['risk_groups'].isin(['0','1'])] # Internalizing vs. healthy
reg2 = dmlr.loc[dmlr['risk_groups'].isin(['0','2'])] # High fat mass vs. healthy
reg3 = dmlr.loc[dmlr['risk_groups'].isin(['0','3'])] # Multimorbid   vs. healthy

# recode dummies 2 and 3 into a 1 to scale 
reg2['nrisk'] = [1 if x == '2' else 0 for x in reg2['risk_groups']]
reg3['nrisk'] = [1 if x == '3' else 0 for x in reg3['risk_groups']]

# Define a funtion to plot logisti regression lines with 95% CI for each group
def logplot(stress, outc, dset, ax, color, label):
    sns.regplot(x = stress, y = outc, data = dset.astype(float), logistic = True, ax = ax, scatter = False,
            color = color, truncate = False, label = label, ci = 95, marker = 'None')

In [ ]:
# Set up a figure for the plots
f, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 10))

# General settings
plt.style.use('bmh')
f.suptitle('Generation R\n', fontsize = 35, fontweight = 'bold')

axs = {'PRENATAL STRESS': ax1, 'POSTNATAL STRESS': ax2}

for key, a in axs.items():
    # adjust the axes limits
    a.set_xlim([-0.1,4.1]) # stress boundaries
    
    pred = "prenatal_stress" if key == 'PRENATAL STRESS' else "postnatal_stress"
    # actual plots (using the function defindes above)  
    logplot(pred, "risk_groups", reg1, a, 'royalblue', 'Internalizing')
    logplot(pred, "nrisk", reg2, a, 'orange', 'High fat mass')
    logplot(pred, "nrisk", reg3, a, 'indianred', 'Multimorbid')
    
    # Axis labels, titles and legend 
    a.set_xlabel('Cumulative Stress Score', fontsize = 25)
    a.set_ylabel('Probabiity', fontsize = 25)
    a.set_title(key, fontsize = 25, fontweight = 'bold')
    a.legend(loc = 'upper left')


In [ ]:
# First define variables coded 1 for each group and 0 for anything else (any other group)
dmlr['p0'] = [1 if x == '0' else 0 for x in dmlr['risk_groups']]
dmlr['p1'] = [1 if x == '1' else 0 for x in dmlr['risk_groups']]
dmlr['p2'] = [1 if x == '2' else 0 for x in dmlr['risk_groups']]
dmlr['p3'] = [1 if x == '3' else 0 for x in dmlr['risk_groups']]

# Set up a figure for the plots
f, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 10))

# General settings
plt.style.use('bmh')
f.suptitle('Generation R\n', fontsize = 35, fontweight = 'bold')

axs = {'PRENATAL STRESS': ax1, 'POSTNATAL STRESS': ax2}

for key, a in axs.items(): 
    # adjust the axes limits
    a.set_xlim([-0.1,4.1]) # stress boundaries
    a.set_ylim([0,1])    # probability boundaries
    
    pred = "prenatal_stress" if key == 'PRENATAL STRESS' else "postnatal_stress"
    # actual plots (using the function defindes above)
    logplot(pred, "p0", dmlr, a, 'seagreen', 'Healthy')
    logplot(pred, "p1", dmlr, a, 'royalblue', 'Internalizing')
    logplot(pred, "p2", dmlr, a, 'orange', 'High fat mass')
    logplot(pred, "p3", dmlr, a, 'indianred', 'Multimorbid')
    
    # Axis labels, titles and legend 
    a.set_xlabel('Cumulative Stress Score', fontsize = 25)
    a.set_ylabel('Probabiity', fontsize = 25)
    a.set_title(key, fontsize = 25, fontweight = 'bold')
    a.legend(loc = 'upper center')
    